# README
**Reminder to Windows and Mac OSX user:**
- auto-sklearn is not or partially supported on these systems, consider using Colab, VM, Docker.
- https://automl.github.io/auto-sklearn/master/installation.html#windows-osx-compatibility  
  
**General workflow for colab user**  
This notebook is organized in multiple steps, grouped in sections and subsections.
1. Set up  
    - install auto-sklearn
    - mount google drive
    - import packages
    - set paths for save/load
2. STEP A - Fetch datasets from OpenML (run once)
    - using sklearn API to fetch (multiple) datasets from OpenML
    - save each dataset as a pickle (.pkl), containing a dict with 'x', 'y' as keys and data as values, named as openml_datasetID.pkl
    - this step is intended to be run once only, afterwards user can load the data from pickle rather than fetch again which is time consuming. If you already have datasets in data folder, skip STEP A.
3. STEP B - Compute meta features (optional)
    - compute the meta features from pickle files, and write result to 'meta_features.pkl'
    - as meta info
    - to test whether the data loading works correctly
    - to test whether the saving result works correctly
    - can skip to STEP C
4. STEP C - Run experiment
    1. Set up
        - set up experiment by choosing budgets, datasets etc.
        - set up autosklearn classifier or modify settings
        - generates seed for reproducbility
        - check estimated time to run experiment, make sure you won't encounter session timeout
    2. Run
        - proceed to start experiment, a nested loop over datasets (outer) and seeds (inner) repeating the procedure 
            - train-test-split
            - train classifier on train set
           - evaluate balanced accuracy on test set
        - generate an unique timestamp id at the beginning
        - results are gathered in a 'res' dict with keys 'cls' and 'acc' containing the trained classifier and test accuracy
    3. Save (recommended)
        - save the 'res' dict to 'experiment_id.pkl' in results 
folder
        - storage warning: file size ~ 1.2GB for 190 trained classifiers + acc, make sure you have sufficient space, else do not save classifiers
        - avoid results loss in colab due to session timeout
        - analyze results in separate notebook to keep running experiment
        - (to do) saving results after each iteration of outer loop

**Particular instruction to colab user:**
- 'Run for colab' contains two cells, each with function to
    1.   install auto-sklearn on colab, then auto restart runtime. Run only once.
    2.   mount google drive for loading/saving datasets and results. In the examples in this notebook, dataset can be load from pickle and results can be dump to pickle, both stored at user-defined directories in google drive.

**Particular instruction to non-colab user:**
- Ignore 'Run for colab'
- Change the paths in 'Set paths' section


**File structure**  
The assumed file structure for the experiment.
  
    directory/
    -- Experiments.ipynb 
    -- Analyze_results.ipynb
    -- data/
    ---- openml_1234.pkl  
    -- results/
    ---- experiments_timestamp/ (note: current this subfolder is not functionable yet)
    ------ experiments_timestamp.pkl
    ------ fig_description_123.pkl

# INCLUDE TO SET UP

## Run for colab
Note: colab will notify warning because the os.kill which restart kernel automatically. User may ignore it and proceed to next cell.

In [ ]:
# For Colab, you need to install auto-sklearn every time
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    !pip install auto-sklearn # Downgrade scipy to 1.4.x
    #!pip install scipy # Upgrade scipy to 1.7.x

import os, signal
os.kill(os.getpid(), signal.SIGKILL) # Restart_runtime

     |████████████████████████████████| 6.4 MB 4.7 MB/s 
     |████████████████████████████████| 38.1 MB 3.1 MB/s 
     |████████████████████████████████| 22.3 MB 7.0 MB/s 
     |████████████████████████████████| 1.1 MB 36.3 MB/s 
     |████████████████████████████████| 830 kB 43.9 MB/s 
     |████████████████████████████████| 4.3 MB 45.1 MB/s 
     |████████████████████████████████| 4.0 MB 42.0 MB/s 
     |████████████████████████████████| 208 kB 63.2 MB/s 
     |████████████████████████████████| 596 kB 49.7 MB/s 
     |████████████████████████████████| 133 kB 61.7 MB/s 
     |████████████████████████████████| 45 kB 3.8 MB/s 
  Created wheel for auto-sklearn: filename=auto_sklearn-0.14.5-py3-none-any.whl size=6602657 sha256=4de52512b362a73011bcf413132daabca644867de779c6a94d62c95ebaf5df21
  Stored in directory: /root/.cache/pip/wheels/90/3f/13/ae0007c007ec02c7ce8ed6cad73c99453d9dfc9c557510b5e9
  Created wheel for pynisher: filename=pynisher-0.6.4-py3-none-any.whl size=7043 sha256=b74a6

In [1]:
import sys
IN_COLAB = 'google.colab' in sys.modules

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Run for everyone

In [2]:
# Dependencies
# Common
import os
import pickle
import sys
import time
from tqdm.autonotebook import tqdm
from datetime import datetime

import numpy as np
import pandas as pd

# Plot
import matplotlib.pyplot as plt
import seaborn as sns

# ML
import sklearn # Import sklearn before autosklearn, solve scipy version error
from sklearn.model_selection import train_test_split
import sklearn.datasets
from sklearn.metrics import accuracy_score, balanced_accuracy_score

# AML
import autosklearn
import autosklearn.classification
from autosklearn.metrics import balanced_accuracy, precision, recall

# Check machine
import multiprocessing
multiprocessing.cpu_count()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  import sys


2

## Set paths

In [3]:
# Set up paths

# User insert folder to store pickles
# dataset name in format openml_xxx.pkl
datasets_folder = '/content/drive/My Drive/Colab Notebooks/course_AML_proj/data'
results_folder = '/content/drive/My Drive/Colab Notebooks/course_AML_proj/results'


# Save result and trained model to pickle for persitence
def save_res(
    res, exp_id,
    results_folder=results_folder
    ):
    # Save
    file_dir = results_folder
    file_path = os.path.join(file_dir, exp_id + '.pkl')
    file_write = open(file_path, "wb")
    # "wb" mode opens the file in binary format for writing
    pickle.dump(res, file_write)
    file_write.close()

# List all classifiers in auto-sklearn
https://automl.github.io/auto-sklearn/master/examples/40_advanced/example_interpretable_models.html#sphx-glr-examples-40-advanced-example-interpretable-models-py

In [ ]:
from autosklearn.pipeline.components.classification import ClassifierChoice

for name in ClassifierChoice.get_components():
    print(name)

adaboost
bernoulli_nb
decision_tree
extra_trees
gaussian_nb
gradient_boosting
k_nearest_neighbors
lda
liblinear_svc
libsvm_svc
mlp
multinomial_nb
passive_aggressive
qda
random_forest
sgd


# WORKFLOW


## STEP A: Fetch dataset and store in pickle (run once)
sklearn fetch from openml https://scikit-learn.org/stable/datasets/loading_other_datasets.html

Error: dataset 41147 gave internal server error

In [ ]:
# Fetch OpenML datasets

# All dataset ids are to be downloaded 
# except 41147 that returned error
# Dataset ids extracted from  Appendix D, Table 20 from the autosklearn2.0 paper
dataset_ids = [41165, 41161, 41159, 41163,
               41142, 1468, 41164, 40996, 
               1111, 12, 41166, 41138,
               41143, 1486, 
               #41147,  # exclude due to internal server error
               41167, 
               41168, 1596, 41150, 40668,
               3, 23512, 41169, 1067,
               23517, 31, 41146, 40984,
               54, 1461, 40981, 1590,
               4135, 40685, 1169, 40975,
               41027, 1489, 1464]

# User insert folder to store pickles
folder_path = datasets_folder

# Fetch and save each dataset
for dataset_id in dataset_ids:
    print(f'Fetching data: {dataset_id}')
    # Using fetch function from sklearn
    x, y = sklearn.datasets.fetch_openml(data_id=dataset_id, 
                                         return_X_y=True, 
                                         as_frame=True) # Return dataframe
    # Store as dict
    data = {'x': x, 'y': y}
    # Dump dict to pickle
    file_to_dump = data

    fname = os.path.join(folder_path, 'openml_' + str(dataset_id))
    file_write = open(fname, 'wb')
    # "wb" mode opens the file in binary format for writing
    pickle.dump(file_to_dump, file_write)
    file_write.close()

## STEP B: Test data loading and compute meta features of test dataset, test result saving 
- Compute meta features of all datasets.
- Also serves as checking whether data can be successfully loaded
- Takes < 1 min to run

In [ ]:
meta_features = {}

folder_path = datasets_folder
all_dataset_names = os.listdir(folder_path)

print(f'Total # datasets = {len(all_dataset_names)}')

for dataset_name in all_dataset_names:

    # Load data
    fname = os.path.join(folder_path, dataset_name)
    file_read = open(fname, "rb")
    data = pickle.load(file_read) # Load pickle to data
    file_read.close()

    # dataset name in format openml_xxx.pkl
    # extract xxx as key
    dataset_id = dataset_name.replace('_', '.').split('.')[1]

    x, y = data['x'], data['y']
    print(f'\nCompute meta features of test dataset: {dataset_id}')
    print(f'meta feature: # instance {x.shape[0]}, # feature {x.shape[1]}')
    meta_features[str(dataset_id)] = {'n_instance': x.shape[0], # Row
                                      'n_feature': x.shape[1], # Col
                                      'n_class': len(np.unique(y))}    

Total # datasets = 38

Compute meta features of test dataset: 41168
meta feature: # instance 83733, # feature 54

Compute meta features of test dataset: 1596
meta feature: # instance 581012, # feature 54

Compute meta features of test dataset: 41150
meta feature: # instance 130064, # feature 50

Compute meta features of test dataset: 40668
meta feature: # instance 67557, # feature 42

Compute meta features of test dataset: 3
meta feature: # instance 3196, # feature 36

Compute meta features of test dataset: 23512
meta feature: # instance 98050, # feature 28

Compute meta features of test dataset: 41169
meta feature: # instance 65196, # feature 27

Compute meta features of test dataset: 1067
meta feature: # instance 2109, # feature 21

Compute meta features of test dataset: 23517
meta feature: # instance 96320, # feature 21

Compute meta features of test dataset: 31
meta feature: # instance 1000, # feature 20

Compute meta features of test dataset: 41146
meta feature: # instance 5124, #

In [ ]:
# Save meta features
folder_path = results_folder

# Dump dict to pickle
file_to_dump = meta_features
fname = os.path.join(folder_path, 'openml_meta_features.pkl')
file_write = open(fname, 'wb')
# "wb" mode opens the file in binary format for writing
pickle.dump(file_to_dump, file_write)
file_write.close()

## STEP C: Run experiments

### Experiment - Baseline

#### Set up experiment
All hyperparameters are default.

Summary of key set up:
- exp id = [40981, 54, 1468, 12, 40984, 3, 1489, 41164, 41165, 41161, 1486, 1461, 1111]
- time budget = 600s (10 min)
- **memory budget = None** (to see if mem budget influence choice of algorithm)
- repeat = 1

In [ ]:
# Auto-sklearn hyperparameters
# Time limit in seconds for the search of appropriate models.
time_left_for_this_task = 600 # Time budget
#per_run_time_limit = int(time_left_for_this_task / 10) # Default is budget/10. Longer the more training time.
# Memory limit in MB for the machine learning algorithm.
memory_limit = None # Memory budget

# Repeat evaluation with different seeds
repeat = 1

# Set random seed generator
rng = np.random.default_rng(12345) # Fix seed for reproducibility
seeds = rng.integers(0, 2**32-1, repeat) # Max seed limited by autoskl

seed_split = 12345 # Fix seed for reproducibility on train test split

# Dataset

# User insert folder to store pickles
folder_path = datasets_folder
# Datasets D_new to evaluate
# All datasets located in the datasets_folder directory set up by user

eval_all_datasets = False

if eval_all_datasets:
    # All datasets in dataset folder
    all_dataset_names = os.listdir(folder_path)
    #all_dataset_names = all_dataset_names[24:]
else:
    # Subset of datasets indicated below
    # Subset below has better performance for shorter time budget
    subset = ['40981', '54', '1468', '12', '40984', '3', '1489', '41164', '41165', '41161', '1486', '1461', '1111']
    all_dataset_names = ['openml_' + name + '.pkl' for name in subset]


# Check estimated experiment duration before proceed
print(f'Est. time required: {repeat * time_left_for_this_task * len(all_dataset_names) / 60} min')
print(f'seeds: {seeds}')

Est. time required: 130.0 min
seeds: [3003105692]


#### Start experiment

In [ ]:
# Run experiment

# Set unique identifier
timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
exp_id = "experiment_" + timestamp

# Tqdm bars
outter_bar = tqdm(all_dataset_names, desc='dataset', leave=False)
inner_bar = tqdm(seeds, desc='seed', leave=False)

# Store result
res = {}

# Start experiment
print(f'\nStart running experiment id: {exp_id}')
start = time.perf_counter()

for dataset_name in all_dataset_names:
    
    outter_bar.update(1) # tqdm progress bar + 1

    # Load data
    fname = os.path.join(folder_path, dataset_name)
    file_read = open(fname, "rb")
    data = pickle.load(file_read) # Load pickle to data
    file_read.close()

    # dataset name in format openml_xxx.pkl
    # extract xxx as key
    dataset_id = dataset_name.replace('_', '.').split('.')[1]

    x, y = data['x'], data['y']
    print(f'\nStart training on dataset: {dataset_id}')
    print(f'meta feature: # instance = {x.shape[0]}, # feature = {x.shape[1]}')

    # Split data
    x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=0.2, random_state=seed_split)

    res_cls = []
    res_acc = []

    for seed in seeds:

        inner_bar.update(1) # tqdm progress bar + 1

        # Train
        cls = autosklearn.classification.AutoSklearnClassifier(
            time_left_for_this_task=time_left_for_this_task,
#            per_run_time_limit=per_run_time_limit, # Default 1/10 of budget
            seed=int(seed),
            memory_limit=memory_limit,
            metric=balanced_accuracy, # For optim
            scoring_functions=[precision, recall] # Not for optim
        )
        # Feurer et al.(2021): single metric for binary clf, multiclass clf and unbalanced datasets

        # Train
        cls.fit(x_train, y_train, x_test, y_test,
                dataset_name=dataset_id)
        
        # Test on trained classfier
        predictions = cls.predict(x_test)
        test_acc = balanced_accuracy_score(y_test, predictions)

        # Store
        res_cls.append(cls)
        res_acc.append(test_acc)

    
    inner_bar.reset() # reset the seed progress bar for next dataset

    res[dataset_id] = {'cls': res_cls, 'acc': res_acc}

    # Save after each dataset finished testing, to avoid mid way interruption
    save_res(res, exp_id)

# Finish experiment
end = time.perf_counter()

print(f'\nExperiment completed in: {(end-start)/60:.1f} min')

dataset:   0%|          | 0/13 [00:00<?, ?it/s]

seed:   0%|          | 0/1 [00:00<?, ?it/s]


Start running experiment id: experiment_20220118-225339

Start training on dataset: 40981
meta feature: # instance = 690, # feature = 14

Start training on dataset: 54
meta feature: # instance = 846, # feature = 18

Start training on dataset: 1468
meta feature: # instance = 1080, # feature = 856

Start training on dataset: 12
meta feature: # instance = 2000, # feature = 216

Start training on dataset: 40984
meta feature: # instance = 2310, # feature = 18

Start training on dataset: 3
meta feature: # instance = 3196, # feature = 36

Start training on dataset: 1489
meta feature: # instance = 5404, # feature = 5

Start training on dataset: 41164
meta feature: # instance = 8237, # feature = 800

Start training on dataset: 41165
meta feature: # instance = 10000, # feature = 7200

Start training on dataset: 41161
meta feature: # instance = 20000, # feature = 4296

Start training on dataset: 1486
meta feature: # instance = 34465, # feature = 118

Start training on dataset: 1461
meta feature:

/usr/local/lib/python3.7/dist-packages/autosklearn/data/feature_validator.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = X[column].astype('category')
/usr/local/lib/python3.7/dist-packages/autosklearn/data/feature_validator.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = X[column].astype('category')
/usr/local/lib/python3.7/dist-packages/autosklearn/data/feature_validator.py:209: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.



Experiment completed in: 131.3 min


#### Saving trained classifiers, test accuracy to pickle
WARNING: file size may be large
1.12GB

In [ ]:
# Save result and trained model to pickle for persitence
save_res(res, exp_id)

#### Logs

### Experiment 1 - Test ensemble size

#### Set up experiment
- subset of 39 data sets are used in this experiment
- rationale of subset: 
- criteria of subset:

- hyperparameter for grid search = test_ensemble_size

Summary of key set up:
- test_ensemble_size = [10, 25, 50] #why setting to 0 doesn't work?

- exp id = [40981, 54, 1468, 12, 40984, 3, 1489, 41164, 41165, 41161, 1486, 1461, 1111]
- time budget = 600s (10 min)
- **memory budget = None** (to see if mem budget influence choice of algorithm)
- repeat = 1

In [4]:
# Auto-sklearn hyperparameters

# Hyperparameters to change
test_ensemble_size = [75, 100]

# Time limit in seconds for the search of appropriate models.
time_left_for_this_task = 600 # Time budget
#per_run_time_limit = int(time_left_for_this_task / 10) # Default is budget/10. Longer the more training time.
# Memory limit in MB for the machine learning algorithm.
memory_limit = None # Memory budget

# Repeat evaluation with different seeds
repeat = 1

# Set random seed generator
rng = np.random.default_rng(12345) # Fix seed for reproducibility
seeds = rng.integers(0, 2**32-1, repeat) # Max seed limited by autoskl

seed_split = 12345 # Fix seed for reproducibility on train test split

# Dataset

# User insert folder to store pickles
folder_path = datasets_folder
# Datasets D_new to evaluate
# All datasets located in the datasets_folder directory set up by user

eval_all_datasets = False

if eval_all_datasets:
    # All datasets in dataset folder
    all_dataset_names = os.listdir(folder_path)
    #all_dataset_names = all_dataset_names[24:]
else:
    # Subset of datasets indicated below
    # Subset below has better performance for shorter time budget
    subset = ['1461', '1486', '41161']
#    subset = ['40981', '54', '1468', '12', '40984', '3', '1489', '41164', '41165', '41161', '1486', '1461', '1111']
    all_dataset_names = ['openml_' + name + '.pkl' for name in subset]


# Check estimated experiment duration before proceed
print(f'Est. time required: {repeat * time_left_for_this_task * len(all_dataset_names) / 60} min')
print(f'seeds: {seeds}')

Est. time required: 30.0 min
seeds: [3003105692]


#### Start experiment

In [5]:
# Run experiment

# Set unique identifier
timestamp = datetime.now().strftime('%Y%m%d-%H%M%S')
exp_id = "experiment_" + timestamp

# Tqdm bars
outter_bar = tqdm(all_dataset_names, desc='dataset', leave=False)
inner_bar = tqdm(seeds, desc='seed', leave=False)

# Store result
res = {}

# Start experiment
print(f'\nStart running experiment id: {exp_id}')
start = time.perf_counter()

for dataset_name in all_dataset_names:
    
    outter_bar.update(1) # tqdm progress bar + 1

    # Load data
    fname = os.path.join(folder_path, dataset_name)
    file_read = open(fname, "rb")
    data = pickle.load(file_read) # Load pickle to data
    file_read.close()

    # dataset name in format openml_xxx.pkl
    # extract xxx as key
    dataset_id = dataset_name.replace('_', '.').split('.')[1]

    x, y = data['x'], data['y']
    print(f'\nStart training on dataset: {dataset_id}')
    print(f'meta feature: # instance = {x.shape[0]}, # feature = {x.shape[1]}')

    # Split data
    # train-validation-test = 64-16-20
    x_train, x_test, y_train, y_test = train_test_split(
        x, y, 
        test_size=0.2, 
        random_state=seed_split)
    
    # Split train set into train and validation
    x_train_val, x_test_val, y_train_val, y_test_val = train_test_split(
        x_train, y_train, 
        test_size=0.2, 
        random_state=seed_split)
    
    res_cls = []
    res_acc = []

    # For hyperparamter tuning using validation
    res_acc_val = {ensemble_size: [] for ensemble_size in test_ensemble_size}

    for ensemble_size in test_ensemble_size:

        # Train
        cls = autosklearn.classification.AutoSklearnClassifier(
            time_left_for_this_task=time_left_for_this_task,
#            per_run_time_limit=per_run_time_limit, # Default 1/10 of budget
            ensemble_size=ensemble_size,
            memory_limit=memory_limit,
            metric=balanced_accuracy, # For optim
            scoring_functions=[precision, recall] # Not for optim
        )
        # Feurer et al.(2021): single metric for binary clf, multiclass clf and unbalanced datasets

        # Train
        cls.fit(x_train_val, y_train_val,
                dataset_name=dataset_id)
        
        # Test on trained classfier
        predictions = cls.predict(x_test_val)
        test_acc = balanced_accuracy_score(y_test_val, predictions)

        # Store
        res_acc_val[ensemble_size].append(test_acc)


    print(f'Print validation accuracy:')
    print(res_acc_val)

    # Choose the optimal ensemble size based on validation accuracy
    best_ind = np.argmax([np.mean(v) for v in res_acc_val.values()])
    best_ensemble_size = test_ensemble_size[best_ind]

    print(f'Choose hyperparameter: {best_ensemble_size}')

    # Retrain with full train set
    for seed in seeds:

        # Train
        cls = autosklearn.classification.AutoSklearnClassifier(
            time_left_for_this_task=time_left_for_this_task,
#            per_run_time_limit=per_run_time_limit, # Default 1/10 of budget
            ensemble_size=ensemble_size,
            seed=int(seed),
            memory_limit=memory_limit,
            metric=balanced_accuracy, # For optim
            scoring_functions=[precision, recall] # Not for optim
        )
        # Feurer et al.(2021): single metric for binary clf, multiclass clf and unbalanced datasets

        # Train
        cls.fit(x_train, y_train, x_test, y_test,
                dataset_name=dataset_id)
        
        # Test on trained classfier
        predictions = cls.predict(x_test)
        test_acc = balanced_accuracy_score(y_test, predictions)

        # Store
#        res_cls.append(cls)
        res_acc.append(test_acc)

    res[dataset_id] = {'cls': res_cls, 'acc': res_acc,
                       'val_hyperparam': 'ensemble_size',
                       'val_acc': res_acc_val}

    # Save after each dataset finished testing, to avoid mid way interruption
    save_res(res, exp_id + "_ensemble_size_" + str(ensemble_size))

# Finish experiment
end = time.perf_counter()

print(f'\nExperiment completed in: {(end-start)/60:.1f} min')

dataset:   0%|          | 0/3 [00:00<?, ?it/s]

seed:   0%|          | 0/1 [00:00<?, ?it/s]


Start running experiment id: experiment_20220203-170623

Start training on dataset: 1461
meta feature: # instance = 45211, # feature = 16
Print validation accuracy:
{75: [0.8507423736178641], 100: [0.8506633347307315]}
Choose hyperparameter: 75

Start training on dataset: 1486
meta feature: # instance = 34465, # feature = 118
Print validation accuracy:
{75: [0.9657269158549799], 100: [0.9646665702286338]}
Choose hyperparameter: 75

Start training on dataset: 41161
meta feature: # instance = 20000, # feature = 4296
Print validation accuracy:
{75: [0.5], 100: [0.5]}
Choose hyperparameter: 75

Experiment completed in: 89.9 min


#### Saving trained classifiers, test accuracy to pickle
WARNING: file size may be large
1.12GB

In [ ]:
# Save result and trained model to pickle for persitence
save_res(res, exp_id)

#### Logs

experiment_20220119-174007
- 40981, 54  
experiment_20220119-192815
- 1468  
experiment_20220119-201052
- 12  
experiment_20220119-205603
- 40984  
experiment_20220119-220334
- 3
- [WARNING] [2022-01-19 22:10:08,353:smac.runhistory.runhistory2epm.RunHistory2EPM4LogCost] Got cost of smaller/equal to 0. Replace by 0.000010 since we use log cost.   
experiment_20220119-225840
- 1489  
experiment_20220119-235856
- 41164  
experiment_20220120-010129
- 41165  
experiment_20220120-021706
- 41161  
experiment_20220120-090232
- 1486  
experiment_20220120-095056
- 1461  
experiment_20220120-194311
- 1111